In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
brand=pd.read_csv("test_brand.csv")
category=pd.read_csv("test_category.csv")
company=pd.read_csv("test_company.csv")

In [3]:
consumer=pd.read_csv("consumer_total.csv")
offers=pd.read_csv("offers.csv")
train_History=pd.read_csv("testHistory.csv")

In [4]:
train_offer=train_History.merge(offers,on='offer')
train_offer.head()

,id,chain,offer,market,offerdate,category,quantity,company,offervalue,brand
0,12262064,95,1219903,39,2013-06-27,799,1,1076211171,1.5,17286
1,15575309,17,1219903,4,2013-07-05,799,1,1076211171,1.5,17286
2,18985721,4,1219903,1,2013-07-15,799,1,1076211171,1.5,17286
3,20552767,14,1219903,8,2013-06-29,799,1,1076211171,1.5,17286
4,21323129,4,1219903,1,2013-06-28,799,1,1076211171,1.5,17286


In [5]:
trainconsumer=consumer.merge(train_offer,on="id")
trainconsumer.head()

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,offerdate,category,quantity,company,offervalue,brand
0,100100562,4727.999975,1308,201,186,210,260,18,1230218,11,2013-07-27,706,1,104127141,1.0,26189
1,100264302,1668.109986,634,53,141,188,172,20,1221665,7,2013-06-30,7205,1,103700030,1.5,4294
2,1002693986,5326.949981,2220,71,241,249,272,46,1221658,15,2013-06-24,7205,2,103700030,3.0,4294
3,100338169,187.119998,53,6,22,28,28,95,1230218,39,2013-07-27,706,1,104127141,1.0,26189
4,100348187,3955.569978,1255,122,230,195,262,14,1221658,8,2013-06-28,7205,2,103700030,3.0,4294


In [6]:
def diff_days(s1,s2):
    date_format = "%Y/%m/%d"
    a = datetime.datetime.strptime(s1, date_format)
    b = datetime.datetime.strptime(s2, date_format)
    delta = b - a
    return delta.days

In [7]:
def diff_offdays(s1,s2):
    a = datetime.datetime.strptime(s1, "%Y/%m/%d")
    b = datetime.datetime.strptime(s2, "%Y-%m-%d")
    delta = b - a
    return delta.days

In [8]:
consumer_brand=pd.DataFrame()
consumer_brand['id']=brand.id.unique()
consumer_category=pd.DataFrame()
consumer_category['id']=category.id.unique()
consumer_company=pd.DataFrame()
consumer_company['id']=company.id.unique()


In [9]:
#bybrand
def extrafeature(x,table):
    print('start-',x)
    if x=='brand':
        varx=brand
    elif x =='category':
        varx=category
    else:
        varx=company
    unique_id=varx.id.unique()
    for i in range(len(unique_id)):
        selectdata=varx[varx['id']== unique_id[i]].reset_index()
        offerday=trainconsumer.loc[trainconsumer.id ==selectdata.id[0],'offerdate'].values[0] 
        table.loc[i,'id']=unique_id[i]
        table.loc[i,x]=selectdata.loc[0,x]
        times=selectdata.shape[0]
        table.loc[i,'purchasetimes_'+x]=times
        table.loc[i,'total_amount_'+x]=selectdata['amount'].sum()
        table.loc[i,'total_quantit_'+x]=selectdata['quantity'].sum()
        table.loc[i,'purchaseperiod_'+x]=0
        table.loc[i,x+'period']=0
        table.loc[i,'before_30day_time_'+x]=0
        table.loc[i,'before_30day_quan_'+x]=0
        table.loc[i,'before_30day_amount_'+x]=0
        table.loc[i,'before_60day_time_'+x]=0
        table.loc[i,'before_60day_quan_'+x]=0
        table.loc[i,'before_60day_amount_'+x]=0
        table.loc[i,'before_90day_time_'+x]=0
        table.loc[i,'before_90day_quan_'+x]=0
        table.loc[i,'before_90day_amount_'+x]=0
        table.loc[i,'before_180day_time_'+x]=0
        table.loc[i,'before_180day_quan_'+x]=0
        table.loc[i,'before_180day_amount_'+x]=0
     
        
        if times >1:
            sumday=0
            period=0
            for k in range(1,times):
                offergap=diff_offdays(selectdata.loc[k,'date'],offerday)
                timegap=diff_days(selectdata.loc[k-1,'date'],selectdata.loc[k,'date'])
                sumday=sumday+int(timegap)
                period=period+(int(timegap)/(selectdata.loc[k-1,'quantity']))
                if  0 < offergap < 30:
                    table.loc[i,'before_30day_time_'+x]+= 1.0
                    table.loc[i,'before_30day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_30day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if 0 < offergap < 60:
                    table.loc[i,'before_60day_time_'+x]+= 1.0
                    table.loc[i,'before_60day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_60day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if  0 < offergap < 90:
                    table.loc[i,'before_90day_time_'+x]+= 1.0
                    table.loc[i,'before_90day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_90day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                        
                if 0 <  offergap < 180:
                    table.loc[i,'before_180day_time_'+x]+= 1.0
                    table.loc[i,'before_180day_quan_'+x]+= float(selectdata.loc[k,'quantity'])
                    table.loc[i,'before_180day_amount_'+x]+= float(selectdata.loc[k,'amount'])
                           
            if type(sumday)==int and sumday>0:    
                table.loc[i,'purchaseperiod_'+x]=(sumday/(times-1))
            if type(period)==int and period>0:
                table.loc[i,x+'period']=(period/(times-1))
        if i%10000==0:
            print(x+": "+ str(i)+"/"+str(len(unique_id)))
            
  

        

In [11]:
extrafeature('brand',consumer_brand)
extrafeature('company',consumer_company)
extrafeature('category',consumer_category)


start- brand
brand: 0/65050


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars


brand: 10000/65050
brand: 20000/65050
brand: 30000/65050
brand: 40000/65050
brand: 50000/65050
brand: 60000/65050
start- company
company: 0/84127
company: 10000/84127
company: 20000/84127
company: 30000/84127
company: 40000/84127
company: 50000/84127
company: 60000/84127
company: 70000/84127
company: 80000/84127
start- category
category: 0/123563
category: 10000/123563
category: 20000/123563
category: 30000/123563
category: 40000/123563
category: 50000/123563
category: 60000/123563
category: 70000/123563
category: 80000/123563
category: 90000/123563
category: 100000/123563
category: 110000/123563
category: 120000/123563


In [12]:
consumer_brand.to_csv("test_brand.csv")
consumer_company.to_csv("test_company.csv")
consumer_category.to_csv("test_category.csv")

In [16]:
##測試是否計算正確
unique_id=brand.id.unique()
selectdata=brand.loc[brand['id']== unique_id[444],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_brand.loc[444,'purchaseperiod_brand'])

test: 9.875
loop: 9.875


In [17]:
consumer_brand.head()

,id,brand,purchasetimes_brand,total_amount_brand,total_quantit_brand,purchaseperiod_brand,brandperiod,before_30day_time_brand,before_30day_quan_brand,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
0,100029473,4294.0,1.0,6.690000,1.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000
1,100100562,26189.0,39.0,107.400001,43.0,10.631579,0.0,4.0,4.0,14.36,5.0,5.0,17.85,7.0,7.0,25.23,19.0,20.0,55.060001
2,100151656,26189.0,11.0,34.440000,14.0,30.800000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,3.0,3.0,9.490000
3,100156512,102504.0,1.0,7.880000,2.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000
4,100176987,93904.0,1.0,14.990000,1.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000


In [19]:
##consumer_category測試是否計算正確
unique_id=category.id.unique()
selectdata=category.loc[category['id']== unique_id[8787],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_category.loc[8787,'purchaseperiod_category'])

test: 59.0
loop: 59.0


In [20]:
##consumer_company 測試是否計算正確
unique_id=company.id.unique()
selectdata=company.loc[company['id']== unique_id[5555],].reset_index()
sumday=0
period=0
times=selectdata.shape[0]
if times==1:
    print('only once')
else:   
    for k in range(1,times):
        day1=datetime.datetime.strptime(selectdata['date'][k],'%Y/%m/%d')
        day2=datetime.datetime.strptime(selectdata['date'][k-1],'%Y/%m/%d')
        sumday=sumday+int((day1-day2).days)
        period=period+(int((day1-day2).days))/selectdata['quantity'][k-1]
    print("test:",sumday/(times-1))
print("loop:",consumer_company.loc[5555,'purchaseperiod_company'])

test: 22.2
loop: 22.2


In [21]:
ccom=consumer_company.set_index('id')

In [22]:
ccate=consumer_category.set_index('id')


In [23]:
cbrand=consumer_brand.set_index('id')

In [24]:
cc=ccom.join(ccate)
cc=cc.join(cbrand)
cc=cc.fillna(0)

In [25]:
cc.head()

,company,purchasetimes_company,total_amount_company,total_quantit_company,purchaseperiod_company,companyperiod,before_30day_time_company,before_30day_quan_company,before_30day_amount_company,before_60day_time_company,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
id,,,,,,,,,,,,,,,,,,,,,
100025687,103700030.0,1.0,1.890000,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100027361,103700030.0,11.0,135.449997,15.0,36.300000,0.0,0.0,0.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100029473,103700030.0,20.0,208.759996,26.0,21.157895,0.0,5.0,8.0,65.919998,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100050424,103700030.0,11.0,186.449998,16.0,26.000000,0.0,1.0,1.0,15.990000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100077437,103700030.0,5.0,17.650000,7.0,43.750000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:

traindata=trainconsumer.join(cc,rsuffix="_firstpurchase",on="id").fillna(0)

In [28]:
traindata[traindata.before_60day_time_brand!=0].head()

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
0,100100562,4727.999975,1308,201,186,210,260,18,1230218,11,...,14.36,5.0,5.0,17.850000,7.0,7.0,25.230000,19.0,20.0,55.060001
12,1007665964,1788.080002,860,41,251,192,241,214,1230218,8,...,0.00,1.0,1.0,1.500000,1.0,1.0,1.500000,1.0,1.0,1.500000
15,101115984,1643.849991,686,66,134,168,157,17,1230218,4,...,3.49,2.0,2.0,6.980000,4.0,4.0,13.960000,7.0,7.0,21.890000
27,101912752,3784.649951,953,110,166,177,219,15,1190530,9,...,21.98,3.0,3.0,32.969999,3.0,3.0,32.969999,3.0,3.0,32.969999
30,102080963,10080.139950,3529,313,341,339,479,18,1213242,11,...,4.99,1.0,1.0,4.990000,1.0,1.0,4.990000,1.0,1.0,4.990000


In [29]:
traindata.columns

Index(['id', 'total_amount', 'total_quantity', 'total_trips', 'diffcompany',
       'diffcategory', 'diffbrand', 'chain', 'offer', 'market', 'offerdate',
       'category', 'quantity', 'company', 'offervalue', 'brand',
       'company_firstpurchase', 'purchasetimes_company',
       'total_amount_company', 'total_quantit_company',
       'purchaseperiod_company', 'companyperiod', 'before_30day_time_company',
       'before_30day_quan_company', 'before_30day_amount_company',
       'before_60day_time_company', 'before_60day_quan_company',
       'before_60day_amount_company', 'before_90day_time_company',
       'before_90day_quan_company', 'before_90day_amount_company',
       'before_180day_time_company', 'before_180day_quan_company',
       'before_180day_amount_company', 'category_firstpurchase',
       'purchasetimes_category', 'total_amount_category',
       'total_quantit_category', 'purchaseperiod_category', 'categoryperiod',
       'before_30day_time_category', 'before_30day_quan

In [30]:
list=['category','brand','company']
for k in list:
    col=k+'_firstpurchase'
    for i in range(traindata.shape[0]):
        if traindata.loc[i,col]==0:
            traindata.loc[i,col]=1
        else:
            traindata.loc[i,col]=0

In [31]:
traindata.to_csv('sum_test.csv')

In [33]:
consumer=pd.read_csv('sum_test.csv')


In [34]:
consumer.columns

Index(['Unnamed: 0', 'id', 'total_amount', 'total_quantity', 'total_trips',
       'diffcompany', 'diffcategory', 'diffbrand', 'chain', 'offer', 'market',
       'offerdate', 'category', 'quantity', 'company', 'offervalue', 'brand',
       'company_firstpurchase', 'purchasetimes_company',
       'total_amount_company', 'total_quantit_company',
       'purchaseperiod_company', 'companyperiod', 'before_30day_time_company',
       'before_30day_quan_company', 'before_30day_amount_company',
       'before_60day_time_company', 'before_60day_quan_company',
       'before_60day_amount_company', 'before_90day_time_company',
       'before_90day_quan_company', 'before_90day_amount_company',
       'before_180day_time_company', 'before_180day_quan_company',
       'before_180day_amount_company', 'category_firstpurchase',
       'purchasetimes_category', 'total_amount_category',
       'total_quantit_category', 'purchaseperiod_category', 'categoryperiod',
       'before_30day_time_category', 'bef

In [35]:
consumer=consumer.drop(columns=['Unnamed: 0'])
consumer.head()

,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,...,before_30day_amount_brand,before_60day_time_brand,before_60day_quan_brand,before_60day_amount_brand,before_90day_time_brand,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand
0,100100562,4727.999975,1308,201,186,210,260,18,1230218,11,...,14.36,5.0,5.0,17.85,7.0,7.0,25.23,19.0,20.0,55.060001
1,100264302,1668.109986,634,53,141,188,172,20,1221665,7,...,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000
2,1002693986,5326.949981,2220,71,241,249,272,46,1221658,15,...,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000
3,100338169,187.119998,53,6,22,28,28,95,1230218,39,...,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000
4,100348187,3955.569978,1255,122,230,195,262,14,1221658,8,...,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.000000


In [36]:
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_company']>1000 or consumer.loc[i,'total_amount_company']<0:
        consumer.loc[i,'total_amount_company']=consumer['total_amount_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_company']>1000 or consumer.loc[i,'total_quantit_company']<0:
        consumer.loc[i,'total_quantit_company']=consumer['total_quantit_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_category']>1000 or consumer.loc[i,'total_amount_category']<0:
        consumer.loc[i,'total_amount_category']=consumer['total_amount_category'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_category']>1000 or consumer.loc[i,'total_quantit_category']<0:
        consumer.loc[i,'total_quantit_category']=consumer['total_quantit_category'].mean()   
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount_brand']>1000 or consumer.loc[i,'total_amount_brand']<0:
        consumer.loc[i,'total_amount_brand']=consumer['total_amount_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantit_brand']>1000 or consumer.loc[i,'total_quantit_brand']<0:
        consumer.loc[i,'total_quantit_brand']=consumer['total_quantit_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_amount']>10000 or consumer.loc[i,'total_amount']<0:
        consumer.loc[i,'total_amount']=consumer['total_amount'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'total_quantity']>10000 or consumer.loc[i,'total_quantity']<0:
        consumer.loc[i,'total_quantity']=consumer['total_quantity'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'categoryperiod']<0 :
        consumer.loc[i,'categoryperiod']=0
for i in range(consumer.shape[0]):
    if consumer.loc[i,'brandperiod']<0:
        consumer.loc[i,'brandperiod']=0
for i in range(consumer.shape[0]):
    if consumer.loc[i,'companyperiod']<0:
        consumer.loc[i,'companyperiod']=0   
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_brand']<=0:
        consumer.loc[i,'purchaseperiod_brand']=consumer['purchaseperiod_brand'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_company']<=0:
        consumer.loc[i,'purchaseperiod_company']=consumer['purchaseperiod_company'].mean()
for i in range(consumer.shape[0]):
    if consumer.loc[i,'purchaseperiod_category']<=0:
        consumer.loc[i,'purchaseperiod_category']=consumer['purchaseperiod_category'].mean()

In [38]:
consumer['ave_spend']=consumer['total_amount']/consumer['total_trips']
consumer['ave_quant']=consumer['total_quantity']/consumer['total_trips']
consumer['ave_unique_company_bought']=consumer['diffcompany']/consumer['total_trips']
consumer['ave_unique_category_bought']=consumer['diffcategory']/consumer['total_trips']
consumer['ave_unique_brand_bought']=consumer['diffbrand']/consumer['total_trips']

In [39]:
print(consumer.shape)
consumer.head()

(151486, 75)


,id,total_amount,total_quantity,total_trips,diffcompany,diffcategory,diffbrand,chain,offer,market,...,before_90day_quan_brand,before_90day_amount_brand,before_180day_time_brand,before_180day_quan_brand,before_180day_amount_brand,ave_spend,ave_quant,ave_unique_company_bought,ave_unique_category_bought,ave_unique_brand_bought
0,100100562,4727.999975,1308.0,201,186,210,260,18,1230218,11,...,7.0,25.23,19.0,20.0,55.060001,23.522388,6.507463,0.925373,1.044776,1.293532
1,100264302,1668.109986,634.0,53,141,188,172,20,1221665,7,...,0.0,0.00,0.0,0.0,0.000000,31.473773,11.962264,2.660377,3.547170,3.245283
2,1002693986,5326.949981,2220.0,71,241,249,272,46,1221658,15,...,0.0,0.00,0.0,0.0,0.000000,75.027465,31.267606,3.394366,3.507042,3.830986
3,100338169,187.119998,53.0,6,22,28,28,95,1230218,39,...,0.0,0.00,0.0,0.0,0.000000,31.186666,8.833333,3.666667,4.666667,4.666667
4,100348187,3955.569978,1255.0,122,230,195,262,14,1221658,8,...,0.0,0.00,0.0,0.0,0.000000,32.422705,10.286885,1.885246,1.598361,2.147541


In [41]:
consumer.to_csv('sum_test.csv')